In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import sys
from pathlib import Path

parent = str(Path().absolute().parent)
if parent not in sys.path:
    sys.path.insert(0,parent)
from sherlock.evidence_store import EvidenceStore, GmailEvidenceStore
from sherlock.models import Claim, Evidence, Argument, EvidenceCollection
from sherlock.agents import ClaimInvestigationAgent
from sherlock.utils import export_argdown
from sherlock.gmail import create_service
import pyperclip

In [ ]:
store = EvidenceStore("wally_evidence")

# Add our evidence
evidence_ids = []
evidence_ids.append(store.add_evidence("Librarian Jane saw someone matching Wally's description at 3pm"))
evidence_ids.append(store.add_evidence("Security camera shows someone in red and white stripes in the park"))
evidence_ids.append(store.add_evidence("Library card records don't show Wally checking out any books today"))
evidence_ids.append(store.add_evidence("Wally's phone GPS shows he was at the library coordinates"))
evidence_ids.append(store.add_evidence("Wally always wears a read and white striped top"))



In [ ]:


agent_pro = ClaimInvestigationAgent(store, supports=True)
agent_con = ClaimInvestigationAgent(store, supports=False)
# Create a test claim
wally_claim = Claim(text="Wally is in the library")


In [ ]:
wally_claim = agent_pro.evaluate_claim(wally_claim)
wally_claim = agent_con.evaluate_claim(wally_claim)
# Evaluate it
wally_claim.likelihood

In [ ]:
arg = export_argdown(wally_claim)

In [ ]:
with open('argdown.txt', 'w') as f:
    f.write(arg)

In [ ]:
wally_park_claim = Claim(text="Wally is in the park")
wally_park_claim = agent_pro.evaluate_claim(wally_park_claim)
wally_park_claim = agent_con.evaluate_claim(wally_park_claim)
arg_park = export_argdown(wally_park_claim)
with open('argdown_park.txt', 'w') as f:
    f.write(arg_park)

In [ ]:
agent_pro.query_evidence('wally park')

In [4]:
# Initialize Gmail evidence store (will prompt for authentication on first use)
try:
    gmail_store = GmailEvidenceStore()
    print("Gmail store initialized successfully!")
except FileNotFoundError as e:
    print(f"Gmail setup needed: {e}")
    gmail_store = None


10:42:11 | INFO | 🔐 [AUTH] Initializing Gmail service using proven create_service() function...


10:42:14 | INFO | ✅ [AUTH] Gmail service created successfully using create_service()


🔐 [AUTH] Credentials loaded from storage successfully
🔐 [AUTH] Required scopes: ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.send', 'https://www.googleapis.com/auth/gmail.modify', 'https://www.googleapis.com/auth/calendar']
🔍 [AUTH] Stored credential scopes: ['https://www.googleapis.com/auth/gmail.modify', 'https://www.googleapis.com/auth/calendar']
🔐 [AUTH] OAuth2 credentials created
✅ [AUTH] Credentials are valid, no refresh needed
✅ [AUTH] Gmail API service built successfully


10:42:14 | INFO | ✅ [AUTH] Gmail API test successful. Email: andylaing5@gmail.com


Gmail store initialized successfully!


In [5]:
# Test Gmail search (only if gmail_store is available)
if gmail_store:
    # Search for recent emails about meetings
    email_results = gmail_store.query("booking AROUND hotel after:2025-07-01")
    print(f"Found {len(email_results)} email results")
    
    # Show first result
    if email_results:
        print("\nFirst email result:")
        print(f"Snippet: {email_results[0].get('snippet', '')}")
        print(f"Text preview: {email_results[0]['text'][:200]}...")
        if 'metadata' in email_results[0]:
            print(f"Subject: {email_results[0]['metadata']['subject']}")
            print(f"From: {email_results[0]['metadata']['sender']}")
else:
    print("Gmail store not available. Please set up credentials first.")


18:41:37 | INFO | 🔍 [GMAIL] Searching Gmail with query: booking AROUND hotel after:2025-07-01
18:41:37 | INFO | 📥 [GMAIL] Adding evidence: Email
Subject: Your Travelodge booking confirmation
Date: Sun, 20 Jul 2025 15:41:25 +0000 (UTC)
Sender: Travelodge Website <webmaster@mail.travelodge.co.uk>
Snippet: YOUR BOOKING CONFIRMATION Hi Andrew Laing, Your booking is now confirmed, please che...
18:41:37 | INFO | 📥 [GMAIL] Adding evidence: Email
Subject: Your Travelodge booking confirmation
Date: Fri, 04 Jul 2025 19:28:32 +0000 (UTC)
Sender: Travelodge Website <webmaster@mail.travelodge.co.uk>
Snippet: YOUR BOOKING CONFIRMATION Hi Andrew Laing, Your booking is now confirmed, please che...
18:41:37 | INFO | ✅ [GMAIL] Found 2 email results for query: booking AROUND hotel after:2025-07-01


Found 2 email results

First email result:
Snippet: 
Text preview: Email
Subject: Your Travelodge booking confirmation
Date: Sun, 20 Jul 2025 15:41:25 +0000 (UTC)
Sender: Travelodge Website <webmaster@mail.travelodge.co.uk>
Snippet: YOUR BOOKING CONFIRMATION Hi Andre...


In [5]:
if gmail_store:
    # Create agents that use Gmail as evidence source
    gmail_agent_pro = ClaimInvestigationAgent(gmail_store, supports=True, max_iterations=10)
    gmail_agent_con = ClaimInvestigationAgent(gmail_store, supports=False, max_iterations=10)
    
    # Example claim about your email habits
    email_claim = Claim(text="Andy travelled abroad in 2025")
    
    print("Evaluating claim using Gmail evidence...")
    # Note: This will search your actual Gmail for evidence
    email_claim = gmail_agent_pro.evaluate_claim(email_claim)
    #email_claim = gmail_agent_con.evaluate_claim(email_claim)
    
    # print(f"Claim likelihood: {email_claim.likelihood}")

else:
    print("Gmail store not available for agent testing.")


10:42:22 | INFO | Starting evaluation of claim for supporting evidence: Andy travelled abroad in 2025
10:42:22 | INFO | Max iterations set to: 10
10:42:22 | INFO | Starting iteration 1/10


Evaluating claim using Gmail evidence...


10:42:26 | INFO | Tool_use stop reason. Response.content: [ToolUseBlock(id='toolu_01YAEJyUA7NQYfXEfUfz2sUz', input={'query': 'Andy travel abroad 2025'}, name='query_evidence', type='tool_use')]
10:42:26 | INFO | Querying evidence store with: Andy travel abroad 2025
10:42:26 | INFO | 🔍 [GMAIL] Searching Gmail with query: Andy travel abroad 2025
10:42:26 | INFO | ✅ [GMAIL] Found 0 email results for query: Andy travel abroad 2025
10:42:26 | INFO | result from tool query_evidence: evidence=[], type is: <class 'sherlock.models.EvidenceCollection'>
10:42:26 | INFO | [{'role': 'user', 'content': 'Evaluate this claim <claim>Andy travelled abroad in 2025</claim>. Use the tools to find supporting evidence and create an argument.'}, {'role': 'assistant', 'content': [ToolUseBlock(id='toolu_01YAEJyUA7NQYfXEfUfz2sUz', input={'query': 'Andy travel abroad 2025'}, name='query_evidence', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01YAEJyUA7NQYfXEfUfz2s

In [6]:
email_claim.model_dump()

[autoreload of sherlock.utils failed: Traceback (most recent call last):
  File "c:\Users\andyl\Python\sherlock\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "c:\Users\andyl\Python\sherlock\.venv\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 524, in maybe_reload_module
    new_source_code = f.read()
  File "C:\Users\andyl\AppData\Local\Programs\Python\Python313\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 5585: character maps to <undefined>
]


{'text': 'Andy travelled abroad in 2025',
 'id': 'andy-travelled-abroad-in-2025',
 'arguments': [],
 'likelihood': {'supporting': 0, 'opposing': 0}}

In [ ]:
# Create a claim that might benefit from multiple evidence sources
complex_claim = Claim(text="The project meeting was productive")

print("This claim could be evaluated using:")
print("1. ChromaDB store: Project documents, notes, reports")
print("2. Gmail store: Meeting invites, follow-up emails, action items")
print("\nEach agent can specialize in different types of evidence!")
